In [1]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from scipy.spatial import distance
# Importing the geodesic module from the library 
from geopy.distance import geodesic 

from statistics import mean

import time
import logging

import pandas as pd
import folium #To plot maps of the solution



coords=pd.read_csv("Input_Data_TSP.csv")
print("Los puntos a considerar para la optimziación son:")
PUNTO_DE_INICIO=4 #Variable donde se ingresa el nodo de inicio o "Home", este DEBE ser un valor entero entre 0 - len(coords), el usuario deberia seleccionar esto

#This block assigns lat, long to separate lists
array=coords.values.tolist()
latitudes =[]
longitudes=[]
for i in range (0,len(array)):
    latitudes.append(array[i][0])
    longitudes.append(array[i][1])

basemap = folium.Map(
            location=[mean(latitudes), mean(longitudes)],
            zoom_start=11)


#This block plots the points to be considered
tooltip = 'Informacion'
for i in range (0,len(array)):
    corresponding_icon='truck'
    corresponding_color='blue'
    
    if i==PUNTO_DE_INICIO:
        corresponding_icon='home'
        corresponding_color='green'
    
    folium.Marker(
    location=[latitudes[i], longitudes[i]],
    tooltip=tooltip,
    popup=('Punto: '+str(i)),
    icon=folium.Icon(icon=corresponding_icon, prefix='fa',color=corresponding_color)
    ).add_to(basemap)
basemap

Los puntos a considerar para la optimziación son:


In [2]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)
logger.warning('LOGGER OK & READY')

LOGGER OK & READY


In [3]:
start_time = time.time()
#Se genera la matriz de distancias
#El modelo DEBE utilizar distancias discretas, el solucionador no acepta distancias con punto decimal, las distancias se deben redondear siempre.
print("La matriz de distancia (distancia geodesica) es:\n")
array=coords.values.tolist()
distance_matrix=[]
for i in range(0,len(array)):
    inner_list=[]
    for j in range (0,len(array)):
        orig=tuple(array[i])
        dest=tuple(array[j])
        distancia_i=round(geodesic(orig, dest).m) #se ejecuta el calculo de distancia en METROS redondeada
        inner_list.append(distancia_i)
    distance_matrix.append(inner_list)
print(distance_matrix)
end_time=(time.time() - start_time)
logger.warning("This process took :"+str(end_time/60)+" minutes to complete.")

This process took :0.0005663156509399414 minutes to complete.


La matriz de distancia (distancia geodesica) es:

[[0, 6074, 9920, 10108, 12630, 14156, 16985, 25920, 22713, 26251, 11890, 13193, 12528, 13010], [6074, 0, 3921, 4813, 7205, 9521, 11660, 20536, 18917, 23298, 9242, 10787, 10553, 12201], [9920, 3921, 0, 2386, 3844, 6785, 8116, 16822, 16313, 21162, 8432, 9897, 10038, 12283], [10108, 4813, 2386, 0, 2523, 4780, 6940, 15875, 14301, 18974, 6073, 7519, 7698, 10023], [12630, 7205, 3844, 2523, 0, 3154, 4457, 13371, 12537, 17533, 6460, 7611, 8097, 10659], [14156, 9521, 6785, 4780, 3154, 0, 3585, 12008, 9547, 14410, 4749, 5383, 6143, 8744], [16985, 11660, 8116, 6940, 4457, 3585, 0, 8941, 9355, 14764, 8299, 8738, 9590, 12141], [25920, 20536, 16822, 15875, 13371, 12008, 8941, 0, 9316, 13792, 16059, 15841, 16838, 18966], [22713, 18917, 16313, 14301, 12537, 9547, 9355, 9316, 0, 5496, 10916, 9882, 10791, 11997], [26251, 23298, 21162, 18974, 17533, 14410, 14764, 13792, 5496, 0, 14442, 13063, 13742, 13966], [11890, 9242, 8432, 6073, 6460, 4749, 8299, 1605

In [4]:
start_time = time.time()

#Este lazo encuentra el arco de mayor longitud en toda la matriz
distancia_maxima=0
for i in range(0,len(distance_matrix)):
    for j in range(0,len(distance_matrix[i])):
        if distance_matrix[i][j]>distancia_maxima:
            distancia_maxima=distance_matrix[i][j]
            
distancia_maxima #Esta variable alberga la distancia máxima entre los arcos de la red

"""Input Parameters for the model """
#Estos parametros los definimos afuera del scope de las funciones para poder manejarlos con mayor facilidad
num_vehicles=5 #Definir el numero de vehículos que se desea usar
vehicle_maximum_travel_distance=(len(distance_matrix)*distancia_maxima) #Esto garantiza que cualquier ruta no supera la distancia máxima factible entre todas las rutas, utilizar si no se tiene un límite de distancia establecido
#vehicle_maximum_travel_distance=60000 #USAR SOLO SI SE TIENE UNA RESTRICCIÓN DE DISTANCIA POR CAMIÓN. TENER EN CUENTA QUE ESTA OPCION DA PASO A QUE SE PUEDAN ESCOGER DISTANCIAS MENORES A LAS DE UNA SOLUCION FACTIBLE, CASO EN EL CUAL NO EXISTIRÁ UNA SOLUCIÓN FACTIBLE Y EL ALGORITMO NO TERMINARÁ NUNCA 
depot=PUNTO_DE_INICIO #Referenciado al inicio del programa, cambiar al inicio del programa

"""Vehicles Routing Problem (VRP)."""

SOLUTION_DATAFRAME=pd.DataFrame() #Listas y df donde se guardarán las soluciones
SOLUTION_DATAFRAME['Index']=coords.index
#SOLUTION_DATAFRAME_1=pd.DataFrame()
#SOLUTION_DATAFRAME_2=pd.DataFrame()

SOLUTION_LIST=[]
SOLUTION_LIST_2=[]
CONTADOR_DF_SOLUCION=0

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix #Referirse a las variables definidas arriba
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        #IBP
        SOLUTION_LIST=[] #Inicializa una lista en blanco donde se conservará la ruta de CADA vehiculo
        #IBP
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            #IBP
            SOLUTION_LIST.append(manager.IndexToNode(index))
            SOLUTION_LIST_2=pd.Series(SOLUTION_LIST)
            #IBP
            #print(manager.IndexToNode(index)) #IBP
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
        #if CONTADOR_DF_SOLUCION ==0:
        SOLUTION_DATAFRAME['Route for Vehicle: '+str(vehicle_id)]=SOLUTION_LIST_2 #Añade las soluciones generadas al dataframe de soluciones
          #  SOLUTION_DATAFRAME_2=pd.concat([SOLUTION_DATAFRAME_0, SOLUTION_DATAFRAME_2], axis=1) 
        #else:
         #   SOLUTION_DATAFRAME_1['Route for Vehicle: '+str(vehicle_id)]=SOLUTION_LIST_2
          #  SOLUTION_DATAFRAME_2=pd.concat([SOLUTION_DATAFRAME_2, SOLUTION_DATAFRAME_1], axis=1)
    print('Maximum of the route distances: {}m'.format(max_route_distance))



def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        vehicle_maximum_travel_distance,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('NO SOLUTION')


if __name__ == '__main__':
    main()

end_time=(time.time() - start_time)
logger.warning("This process took :"+str(end_time)+" seconds to complete.")

This process took :0.05129814147949219 seconds to complete.


Route for vehicle 0:
 4 ->  2 ->  1 ->  0 ->  3 -> 4
Distance of the route: 26470m

Route for vehicle 1:
 4 ->  8 ->  7 ->  6 -> 4
Distance of the route: 35251m

Route for vehicle 2:
 4 ->  10 ->  12 ->  13 ->  11 -> 4
Distance of the route: 21748m

Route for vehicle 3:
 4 -> 4
Distance of the route: 0m

Route for vehicle 4:
 4 ->  9 ->  5 -> 4
Distance of the route: 35097m

Maximum of the route distances: 35251m


In [5]:
#Solucion con la ruta
solution_map = folium.Map(
            location=[mean(latitudes), mean(longitudes)],
            zoom_start=11.5)

#All possible colors for markers and lines
icon_color=['red', 'blue', 'purple', 'orange', 'darkred',
            'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
            'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
            'gray', 'black', 'lightgray', 'green']

for i in range(0,(len(SOLUTION_DATAFRAME.columns)-1)):
    current_route=pd.merge(SOLUTION_DATAFRAME,coords,left_on=('Route for Vehicle: '+str(i)),right_on=coords.index)
    current_route

    ordered_sol_list=current_route[['latitude','longitude']].values.tolist()
    plot_sol_list=[]
    for j in range (0,len(ordered_sol_list)):
        coord=tuple(ordered_sol_list[j])
        plot_sol_list.append(coord)

        #points=points_tuple_list
        points=plot_sol_list
        k=0
        for each in points:
    
            corresponding_icon='truck'
            corresponding_color=icon_color[i]
    
            if k==0:
                corresponding_icon='home'
                corresponding_color='green'
    
            folium.Marker(
            each,
            tooltip=tooltip,
            popup=('Punto: '+str(int(SOLUTION_DATAFRAME['Route for Vehicle: '+str(i)][k]))),
            icon=folium.Icon(icon=corresponding_icon, prefix='fa',color=corresponding_color)
            ).add_to(solution_map)
            k+=1
    
            #incluye lineas
            folium.PolyLine(points, color=icon_color[i], weight=2.5, opacity=1).add_to(solution_map)
#solution_map

In [6]:
basemap

In [7]:
solution_map